This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-optimize/abtesting).

## A/B Testing

A/B testing is a method that provides the ability to test out ML models for performance, accuracy or other useful benchmarks.  A/B testing is contrasted with the Wallaroo Shadow Deployment feature.  In both cases, two sets of models are added to a pipeline step:

* Control or Champion model:  The model currently used for inferences.
* Challenger model(s): One or more models that are to be compared to the champion model.

The two feature are different in this way:

| Feature | Description |
|---|---|
| A/B Testing | A subset of inferences are submitted to either the champion ML model or a challenger ML model. |
| Shadow Deploy | All inferences are submitted to the champion model and one or more challenger models. |

So to repeat:  A/B testing submits *some* of the inference requests to the champion model, some to the challenger model with one set of outputs, while shadow testing submits *all* of the inference requests to champion and shadow models, and has separate outputs.

This tutorial demonstrate how to conduct A/B testing in Wallaroo.  For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model and a challenger model.
* Create a pipeline that can ingest our submitted data with the champion model and the challenger model set into a A/B step.
* Run a series of sample inferences to display inferences that are run through the champion model versus the challenger model, then determine which is more efficient.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Prerequisites

* A deployed Wallaroo instance
* The following Python libraries installed:
  * `os`
  * `json`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame

## Steps

### Import libraries

Here we will import the libraries needed for this notebook.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import pandas as pd
import json
from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace for all other commands.

In [3]:
workspace_name = 'abhousetesting'

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

{'name': 'abhousetesting', 'id': 30, 'archived': False, 'created_by': '65124b18-8382-49af-b3c8-ada3b9df3330', 'created_at': '2024-04-16T21:09:02.201859+00:00', 'models': [], 'pipelines': []}

### Set Up the Champion and Challenger Models

Now we upload the Champion and Challenger models to our workspace.  We will use two models:

1. `aloha-cnn-lstm` model.
2. `aloha-cnn-lstm-new` (a retrained version)

### Set the Champion Model

We upload our champion model, labeled as `control`.

In [58]:
control = wl.upload_model("houseprice-control",
                          'models/gbr_model.onnx', 
                          framework=wallaroo.framework.Framework.ONNX) \
                          .configure(tensor_fields=['tensor'])

### Set the Challenger Model

Now we upload the Challenger model, labeled as `challenger`.

In [59]:
challenger = wl.upload_model("houseprice-challenger",
                          'models/rf_model.onnx', 
                          framework=wallaroo.framework.Framework.ONNX) \
                          .configure(tensor_fields=['tensor'])

### Define The Pipeline

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

In [60]:
pipeline.undeploy()
pipeline.clear()
pipeline = (wl.build_pipeline("randomsplitpipeline-demo")
            .add_random_split([(2, control), 
                               (1, challenger)], 
                               "session_id"))

### Deploy the pipeline

Now we deploy the pipeline so we can run our inference through it.

In [61]:
experiment_pipeline = pipeline.deploy()

# Run a single inference
Now we have our deployment set up let's run a single inference. In the results we will be able to see the inference results as well as which model the inference went to under model_id.  We'll run the inference request 5 times, with the odds are that the challenger model being run at least once.

In [62]:
# use dataframe JSON files
for x in range(5):
    result = experiment_pipeline.infer_from_file("./data/xtest-1.df.json")
    value = result.loc[0]["out.variable"]
    # model="test"
    model = json.loads(result.loc[0]["out._model_split"][0])['model_version']['name']
    # display(model)
    # model = json.loads(result.loc[0, ["out._model_split"]])
    df = pd.DataFrame({'model': model, 'value': value})
    display(df)  

,model,value
0,houseprice-control,704901.9


,model,value
0,houseprice-challenger,718013.7


,model,value
0,houseprice-control,704901.9


,model,value
0,houseprice-challenger,718013.7


,model,value
0,houseprice-challenger,718013.7


### Run Inference Batch

We will submit 1000 rows of test data through the pipeline, then loop through the responses and display which model each inference was performed in.  The results between the control and challenger should be approximately 2:1.

In [63]:
responses = []
test_data = pd.read_json('data/normal-inputs.df.json')
# For each row, submit that row as a separate dataframe
# Add the results to the responses array
for index, row in test_data.head(1000).iterrows():
    responses.append(experiment_pipeline.infer(row.to_frame('tensor').reset_index()))

#now get our responses for each row
l = [json.loads(r.loc[0]["out._model_split"][0])['model_version']['name'] for r in responses]
df = pd.DataFrame({'model': l})
display(df.model.value_counts())

houseprice-control       645
houseprice-challenger    355
Name: model, dtype: int64

### Logs

Logs can be viewed with the Pipeline method `logs()`.  For this example, only the first 5 logs will be shown.  For Arrow enabled environments, the model type can be found in the column `out._model_split`.

In [64]:
logs = experiment_pipeline.logs(limit=5)
display(logs.loc[:,['time', 'out._model_split', 'out.variable']])

,time,out._model_split,out.variable
0,2024-04-16 21:40:10.160,"[{""model_version"":{""name"":""houseprice-control"",""visibility"":""private"",""workspace_id"":30,""conversion"":{""arch"":""x86"",""accel"":""none"",""python_version"":""3.8"",""requirements"":[],""framework"":""onnx""},""id"":45,""image_path"":null,""status"":""ready"",""task_id"":null,""file_info"":{""version"":""f92c6130-4c73-4a40-9869-5e4d17d30122"",""sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a"",""file_name"":""gbr_model.onnx""},""created_on_version"":""2024.1.0""},""config"":{""id"":71,""model_version_id"":45,""runtime"":""onnx"",""filter_threshold"":null,""tensor_fields"":[""tensor""],""input_schema"":null,""output_schema"":null,""batch_config"":null,""sidekick_uri"":null}}]",[704901.9]
1,2024-04-16 21:40:09.671,"[{""model_version"":{""name"":""houseprice-control"",""visibility"":""private"",""workspace_id"":30,""conversion"":{""arch"":""x86"",""accel"":""none"",""python_version"":""3.8"",""requirements"":[],""framework"":""onnx""},""id"":45,""image_path"":null,""status"":""ready"",""task_id"":null,""file_info"":{""version"":""f92c6130-4c73-4a40-9869-5e4d17d30122"",""sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a"",""file_name"":""gbr_model.onnx""},""created_on_version"":""2024.1.0""},""config"":{""id"":71,""model_version_id"":45,""runtime"":""onnx"",""filter_threshold"":null,""tensor_fields"":[""tensor""],""input_schema"":null,""output_schema"":null,""batch_config"":null,""sidekick_uri"":null}}]",[704901.9]
2,2024-04-16 21:40:09.513,"[{""model_version"":{""name"":""houseprice-control"",""visibility"":""private"",""workspace_id"":30,""conversion"":{""arch"":""x86"",""accel"":""none"",""python_version"":""3.8"",""requirements"":[],""framework"":""onnx""},""id"":46,""image_path"":null,""status"":""ready"",""task_id"":null,""file_info"":{""version"":""3860eca1-9501-4137-81a5-54f3b96f8322"",""sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"",""file_name"":""rf_model.onnx""},""created_on_version"":""2024.1.0""},""config"":{""id"":73,""model_version_id"":46,""runtime"":""onnx"",""filter_threshold"":null,""tensor_fields"":[""tensor""],""input_schema"":null,""output_schema"":null,""batch_config"":null,""sidekick_uri"":null}}]",[718013.7]
3,2024-04-16 21:40:09.352,"[{""model_version"":{""name"":""houseprice-control"",""visibility"":""private"",""workspace_id"":30,""conversion"":{""arch"":""x86"",""accel"":""none"",""python_version"":""3.8"",""requirements"":[],""framework"":""onnx""},""id"":45,""image_path"":null,""status"":""ready"",""task_id"":null,""file_info"":{""version"":""f92c6130-4c73-4a40-9869-5e4d17d30122"",""sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a"",""file_name"":""gbr_model.onnx""},""created_on_version"":""2024.1.0""},""config"":{""id"":71,""model_version_id"":45,""runtime"":""onnx"",""filter_threshold"":null,""tensor_fields"":[""tensor""],""input_schema"":null,""output_schema"":null,""batch_config"":null,""sidekick_uri"":null}}]",[704901.9]
4,2024-04-16 21:40:09.183,"[{""model_version"":{""name"":""houseprice-control"",""visibility"":""private"",""workspace_id"":30,""conversion"":{""arch"":""x86"",""accel"":""none"",""python_version"":""3.8"",""requirements"":[],""framework"":""onnx""},""id"":45,""image_path"":null,""status"":""ready"",""task_id"":null,""file_info"":{""version"":""f92c6130-4c73-4a40-9869-5e4d17d30122"",""sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a"",""file_name"":""gbr_model.onnx""},""created_on_version"":""2024.1.0""},""config"":{""id"":71,""model_version_id"":45,""runtime"":""onnx"",""filter_threshold"":null,""tensor_fields"":[""tensor""],""input_schema"":null,""output_schema"":null,""batch_config"":null,""sidekick_uri"":null}}]",[704901.9]


### Undeploy Pipeline

With the testing complete, we undeploy the pipeline to return the resources back to the environment.

In [65]:
experiment_pipeline.undeploy()

name,randomsplitpipeline-demo
created,2024-04-16 21:09:05.335685+00:00
last_updated,2024-04-16 21:41:02.996170+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"b3c23fa4-9e31-4ac0-b312-81aab8a27c48, 4cf6ce88-1202-4d2d-8506-285884c8975a, 16d43d1a-7145-4f20-a64d-fd10c0e35913, f5e79bc5-cfef-40f6-a93b-04f1fe8c3366, 0d5684ae-d030-4eed-a218-f207e2a2a2e8, a6522c44-e2c3-4954-8702-bf191b53fa50, dc607540-e04e-467f-a8c9-d112fd9a593d, a2d6d6c3-8690-4f04-bc50-3f21a24fdd2d, fd3c9737-4bae-4106-8b7d-eb4262f19d4c, 388a009b-70fe-4613-b4b3-18c50cb03170, 7f81102b-2e73-434b-ad59-d74b0e4b5436, e38d5d00-4796-45a7-bbec-8fd05fb940d2"
steps,houseprice-control
published,False
